<a href="https://colab.research.google.com/github/babelfishz/ocalendar/blob/master/orchid_gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive	sample_data


In [0]:
!pip install tf-nightly

In [0]:
!pip install --upgrade tensorflow
#!pip install tf-nightly

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Dropout, Input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
train_path = './gdrive/My Drive/dataSet/train'
valid_path = './gdrive/My Drive/dataSet/val'
test_path = './gdrive/My Drive/dataSet/test'

train_batches = ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),  batch_size=32)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),  batch_size=32)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224),  batch_size=32)

Found 188 images belonging to 3 classes.
Found 18 images belonging to 3 classes.
Found 18 images belonging to 3 classes.


In [0]:
vgg16_model = tensorflow.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
for layer in vgg16_model.layers[:-4]:
    layer.trainable = False
    
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(vgg16_model)
 
# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit(train_batches, 
            epochs=5, 
            validation_data=valid_batches, 
            steps_per_epoch=train_batches.samples/train_batches.batch_size,
            validation_steps=valid_batches.samples/valid_batches.batch_size,
            verbose=1)

58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 40,408,899
Trainable params: 32,773,635
Non-trainable params: 7,635,264
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to

In [0]:
test_imgs, test_labels = next(test_batches)

predictions = model.predict(test_imgs)

print(predictions)

In [0]:
tf.keras.models.save_model(model, "./gdrive/My Drive/_models/flower/1")
print("models written")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./gdrive/My Drive/_models/flower/1/assets
models written


In [0]:
saved_model = tf.keras.models.load_model("./gdrive/My Drive/_models/flower/1")
input_shape = (None, 224, 224, 3)
saved_model.build(input_shape)
saved_model.summary()

#imported = tf.saved_model.load("/content/gdrive/My Drive/Models/flower/1")




Two checkpoint references resolved to different objects (<tensorflow.python.keras.engine.training.Model object at 0x7fc28610add8> and <tensorflow.python.keras.layers.core.Flatten object at 0x7fc2861001d0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7fc2861006d8> and <tensorflow.python.keras.layers.core.Dropout object at 0x7fc286100d30>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7fc2861006d8> and <tensorflow.python.keras.layers.core.Flatten object at 0x7fc2861001d0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7fc286104240> and <tensorflow.python.keras.layers.core.Dropout object at 0x7fc286100d30>).
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)               

In [0]:
test_imgs, test_labels = next(test_batches)

predictions = saved_model.predict(test_imgs)

print(predictions)

In [0]:
def preprocess_b64_image(image_bytes, h=224, w=224):
    image = tf.io.decode_base64(image_bytes[0])
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, size=[h, w])
    #image = (image - 127.5) / 127.5
    image = tf.expand_dims(image, axis=0)
    return image

#image_bytes = Input(shape = (1,),dtype="string")
image_bytes = tf.keras.Input(shape=[], batch_size=1, name='b64_image_bytes', dtype=tf.string)
preprocessed_image = preprocess_b64_image(image_bytes)

In [0]:
predictions = saved_model(preprocessed_image)
new_model = tf.keras.Model(image_bytes, predictions)
new_model.summary()
print('Model Input Shape:', new_model.input_shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
b64_image_bytes (InputLayer) [(1,)]                    0         
_________________________________________________________________
tf_op_layer_strided_slice (T [()]                      0         
_________________________________________________________________
tf_op_layer_DecodeBase64 (Te [()]                      0         
_________________________________________________________________
tf_op_layer_DecodeJpeg (Tens [(None, None, 3)]         0         
_________________________________________________________________
tf_op_layer_ExpandDims (Tens [(1, None, None, 3)]      0         
_________________________________________________________________
tf_op_layer_ResizeBilinear ( [(1, 224, 224, 3)]        0         
_________________________________________________________________
tf_op_layer_Squeeze (TensorF [(224, 224, 3)]           0     

In [0]:

def preprocess_and_decode(img_str, new_shape=[224,224]):
    img = tf.io.decode_base64(img_str)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, new_shape, method=tf.image.ResizeMethod.BILINEAR)
    # if you need to squeeze your input range to [0,1] or [-1,1] do it here
    return img
InputLayer = Input(shape = (1,),dtype="string")
OutputLayer = tf.keras.layers.Lambda(lambda img : tf.map_fn(lambda im : preprocess_and_decode(im[0]), img, dtype="float32"))(InputLayer)
base64_model = tf.keras.Model(InputLayer,OutputLayer)


In [0]:
base64_input = base64_model.input
final_output = saved_model(base64_model.output)
new_model = tf.keras.Model(base64_input,final_output)

new_model.summary()

In [0]:
tf.keras.models.save_model(new_model, "./gdrive/My Drive/_models/flower/2")

INFO:tensorflow:Assets written to: ./gdrive/My Drive/_models/flower/2/assets


In [0]:
new_saved_model = tf.keras.models.load_model("./gdrive/My Drive/_models/flower/2")
new_saved_model.summary()

In [0]:
# plots images with labels within jupyter notebook
def plots(ims, figsize=(24,12), rows=6, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=32)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

#imgs, labels = next(train_batches)
#plots(imgs, titles=labels)

In [0]:
!ls -lR ./gdrive/My\ Drive/models

'./gdrive/My Drive/models':
total 8
drwx------ 3 root root 4096 Nov  5 03:19 flower
drwx------ 2 root root 4096 Sep 17 02:35 flowers

'./gdrive/My Drive/models/flower':
total 285951
drwx------ 4 root root      4096 Nov  5 03:19 1
-rw------- 1 root root 292808752 Nov  5 03:19 orchid.h5

'./gdrive/My Drive/models/flower/1':
total 428
drwx------ 2 root root   4096 Nov  5 03:19 assets
-rw------- 1 root root 429474 Nov  5 03:19 saved_model.pb
drwx------ 2 root root   4096 Nov  5 03:19 variables

'./gdrive/My Drive/models/flower/1/assets':
total 0

'./gdrive/My Drive/models/flower/1/variables':
total 285887
-rw------- 1 root root 292744486 Nov  5 03:19 variables.data-00000-of-00001
-rw------- 1 root root      3296 Nov  5 03:19 variables.index

'./gdrive/My Drive/models/flowers':
total 4
drwx------ 2 root root 4096 Sep 17 03:41 00001

'./gdrive/My Drive/models/flowers/00001':
total 430
drwx------ 2 root root   4096 Sep 17 03:41 assets
-rw------- 1 root root 431974 Sep 17 03:41 saved_model.pb


In [0]:
!ls -lR ./gdrive/My\ Drive/_models


In [0]:
!saved_model_cli show --dir ./gdrive/My\ Drive/_models/flower/2 --all

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras


image_bytes = tf.keras.Input(shape=[], batch_size=1, name='b64_image_bytes', dtype=tf.string)

print(image_bytes)
print(image_bytes.shape)

print(image_bytes[0])



Tensor("b64_image_bytes_1:0", shape=(1,), dtype=string)
(1,)
Tensor("strided_slice:0", shape=(), dtype=string)
